# 데이터 로드

- 학습목표
    - 사이킷런의 결정트리를 진행
    
    
- 핵심키워드
    - 사이킷런
    - Decision Tree
    - fit. predict
    - graphviz
    

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('../04_모델과 파라미터 찾기/diabetes_feature.csv')

print(df.shape)
df.head()

(768, 16)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,0,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,0,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,0,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


In [8]:
df_insulin = pd.read_csv('diabetes_fill_insulin.csv')

df['Insulin'] = df_insulin['Insulin']
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,184.855904,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,66.159991,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,194.151175,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94.000000,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168.000000,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


In [9]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [10]:
X = df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high']]

X.shape

(768, 8)

In [11]:
y = df['Outcome']
y.shape

(768,)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X_train.shape, y_train.shape

((614, 8), (614,))

In [18]:
X_test.shape, y_test.shape

((154, 8), (154,))

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42),
              RandomForestClassifier(random_state=42),
              GradientBoostingClassifier(random_state=42)]

estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [20]:
max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.3, 1.0, 10)

In [23]:
results = []
for estimator in estimators:
    result = []
    result.append(estimator.__class__.__name__)
    results.append(result)
results

[['DecisionTreeClassifier'],
 ['RandomForestClassifier'],
 ['GradientBoostingClassifier']]

In [24]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.3, 1.0, 10)

param_distributions = {'max_depth': max_depth,
                       'max_features': max_features}

results = []
for estimator in estimators:
    result = []
    if estimator.__class__.__name__ != 'DecisionTreeClassifier':
        param_distributions['n_estimators'] = np.random.randint(100, 200, 10)
    
    clf = RandomizedSearchCV(estimator,
                             param_distributions,
                             n_iter=100,
                             n_jobs=-1,
                             scoring='accuracy',
                             cv=5,
                             verbose=2)
    
    clf.fit(X_train, y_train)
    
    result.append(estimator.__class__.__name__)
    result.append(clf.best_params_)
    result.append(clf.best_score_)
    result.append(clf.score(X_test, y_test))
    result.append(clf.cv_results_)
    
    results.append(result)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 333 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   14.2s finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 365 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   56.8s finished


In [25]:
df = pd.DataFrame(results,
                  columns = ['estimator', 'best_params', 'train_score', 'test_score', 'cv_result'])
df

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.46053728498643776, 'max_dep...",0.757364,0.772727,"{'mean_fit_time': [0.00702509880065918, 0.0073..."
1,RandomForestClassifier,"{'n_estimators': 148, 'max_features': 0.874276...",0.799707,0.733766,"{'mean_fit_time': [0.38278446197509763, 0.3226..."
2,GradientBoostingClassifier,"{'n_estimators': 155, 'max_features': 0.348209...",0.788351,0.740260,"{'mean_fit_time': [0.10715436935424805, 0.9202..."
